In [1]:
import pandas as pd
import spacy
from Zen2Han import normalize

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report

GiNZA による品詞分解

In [2]:
nlp = spacy.load('ja_ginza_nopn')
def wakati(sentence):
    s = normalize(sentence)
    doc = nlp(s)
    sep_s = ' '.join([t.text for t in doc])
    return sep_s

csv形式のデータをDataFrameで取得

In [3]:
def get_df(name):
    df = pd.read_csv('./data/{}.csv'.format(name), index_col=0, encoding='utf-8-sig')
    return df

DataFrameから質問文(X)と資料番号(y)を抽出

In [4]:
def data_prepro(df):
    X = np.empty((0,1),np.string_)
    y = np.empty((0,1),np.string_)
    for key, row in df.iterrows():
        d,q = row['document'], row['question']
        if not d or not q:
            continue
        sep_q = wakati(q)
        
        X = np.append(X, sep_q)
        y = np.append(y, d)
    return X, y

各データの準備

In [5]:
# training + validation
X_train, y_train = data_prepro(get_df('train_qa'))
X_dev, y_dev = data_prepro(get_df('dev_qa'))
X = np.hstack((X_train, X_dev))
y = np.hstack((y_train, y_dev))
print(X[0:10])
print(y[0:10])
print(X.shape, y.shape)

['黄色い ごみ袋 を 使っ て も いい です か 。' '色 付き の ごみ袋 を 使っ て も いい です か 。'
 '京都市 指定 の ごみ袋 を 用い て いい です か 。' 'ゴミ袋 に は 何色 の もの を 使用 すれ ば いい です か 。'
 '京都市 の 家庭 ごみ 用 の 袋 を 使用 でき ます か 。'
 '灰色 の ごみ袋 を 使っ たら 怒ら れ まし た 。 なぜ でしょう か 。' 'レジ 袋 で 捨て て も いい です か 。'
 '大学 指定 の ごみ袋 は 何 です か 。' '大学 で 使える ゴミ袋 は 何 です か 。'
 'ごみ袋 は どの よう な もの を 使え ば よい です か 。']
[b'1' b'1' b'1' b'1' b'1' b'1' b'1' b'1' b'1' b'1']
(1480,) (1480,)


In [6]:
# test
Xe, ye = data_prepro(get_df('test_qa'))
print(Xe[0:10])
print(ye[0:10])
print(Xe.shape, ye.shape)

['ゴミ袋 は 京都市 の ゴミ袋 です か 。' 'ごみ は 何 時 に 回収 し ます か 。'
 '冷蔵庫 は どこ に 捨てれ ば いい です か 。' '燃やす ゴミ で 捨て られる もの は 何 です か 。'
 '段 ボール は どの よう に 捨てれ ば いい です か 。' 'ペットボトル は どう やっ て 捨てれ ば いい です か 。'
 '産業廃棄物 に 紙 類 は 含ま れ ます か 。' '実験 系 プラスチック と は 何 です か 。'
 '大型 ごみ 集積 所 は どこ です か 。' '大型 ごみ に 蛍光灯 は 含ま れ ます か 。']
[b'1' b'1' b'1' b'1' b'1' b'1' b'1' b'1' b'1' b'1']
(94,) (94,)


## SVMでの学習

In [7]:
clf = SVC(kernel='linear', class_weight = 'balanced')
clf

SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [8]:
labels = [str(i+1) for i in range(10)]
labels

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

## SVM + CountVectorizer

In [9]:
vectorizer = CountVectorizer(min_df=2)
Xv = vectorizer.fit_transform(X)
print(Xv.shape[1])
Xev = vectorizer.transform(Xe)
clf = clf.fit(Xv, y)
print(classification_report(ye, clf.predict(Xev), target_names=labels))

750
              precision    recall  f1-score   support

           1       0.91      1.00      0.95        10
           2       1.00      1.00      1.00        10
           3       1.00      1.00      1.00         9
           4       1.00      1.00      1.00        11
           5       1.00      1.00      1.00        11
           6       0.89      1.00      0.94         8
           7       1.00      0.78      0.88         9
           8       1.00      1.00      1.00         9
           9       1.00      1.00      1.00         7
          10       1.00      1.00      1.00        10

   micro avg       0.98      0.98      0.98        94
   macro avg       0.98      0.98      0.98        94
weighted avg       0.98      0.98      0.98        94



In [10]:
vectorizer = CountVectorizer(min_df=2, max_df=0.3)
Xv2 = vectorizer.fit_transform(X)
print(Xv2.shape[1])
Xev2 = vectorizer.transform(Xe)
clf = clf.fit(Xv2, y)
print(classification_report(ye, clf.predict(Xev2), target_names=labels))

748
              precision    recall  f1-score   support

           1       0.91      1.00      0.95        10
           2       1.00      1.00      1.00        10
           3       1.00      1.00      1.00         9
           4       1.00      1.00      1.00        11
           5       1.00      1.00      1.00        11
           6       0.89      1.00      0.94         8
           7       1.00      0.78      0.88         9
           8       1.00      1.00      1.00         9
           9       1.00      1.00      1.00         7
          10       1.00      1.00      1.00        10

   micro avg       0.98      0.98      0.98        94
   macro avg       0.98      0.98      0.98        94
weighted avg       0.98      0.98      0.98        94



## SVM + tfidf

In [11]:
vectorizer = TfidfVectorizer(min_df=2, max_df=0.3)
Xv2 = vectorizer.fit_transform(X)
print(Xv2.shape[1])
Xev2 = vectorizer.transform(Xe)
clf = clf.fit(Xv2, y)
print(classification_report(ye, clf.predict(Xev2), target_names=labels))

748
              precision    recall  f1-score   support

           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00        10
           3       1.00      1.00      1.00         9
           4       1.00      1.00      1.00        11
           5       1.00      1.00      1.00        11
           6       1.00      1.00      1.00         8
           7       1.00      1.00      1.00         9
           8       1.00      1.00      1.00         9
           9       1.00      1.00      1.00         7
          10       1.00      1.00      1.00        10

   micro avg       1.00      1.00      1.00        94
   macro avg       1.00      1.00      1.00        94
weighted avg       1.00      1.00      1.00        94

